In [1]:
!pip install vllm gradio --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.2/438.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from vllm import LLM, SamplingParams

model_name = "/content/drive/MyDrive/vllm_models/phi2"

sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    max_tokens=256
)

# Initialize the vLLM model
llm = LLM(model=model_name, dtype="float16", gpu_memory_utilization=0.8)


INFO 11-05 13:23:22 [__init__.py:216] Automatically detected platform cuda.
INFO 11-05 13:23:34 [utils.py:233] non-default args: {'dtype': 'float16', 'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'model': '/content/drive/MyDrive/vllm_models/phi2'}
INFO 11-05 13:23:57 [model.py:547] Resolved architecture: PhiForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 11-05 13:23:57 [model.py:1510] Using max model len 2048
INFO 11-05 13:24:00 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 11-05 13:24:03 [__init__.py:3036] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 11-05 13:27:51 [llm.py:306] Supported_tasks: ['generate']


In [3]:
# prompt ="Generate 3 single choice MCQs from the given content"
# content ="""Python is a high-level programming language known for its readability and ease of use.
# It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms,
# including procedural, object-oriented, and functional programming."""
# full_prompt =f"{prompt}\n {content}"



In [6]:
# outputs = llm.generate([full_prompt],sampling_params)
# for out in outputs:
#   print(out.outputs[0].text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


A: Q1. Who created Python?
A. Guido van Rossum
B. Steve Jobs
C. Bill Gates
D. Mark Zuckerberg

Q2. In what year was Python first released?
A. 1980
B. 1991
C. 2005
D. 2010

Q3. What programming paradigms does Python support?
A. Procedural, object-oriented, and functional programming
B. Only procedural programming
C. Only functional programming
D. Only object-oriented programming



In [13]:
def stream_mcqs_from_phi2(prompt):
    try:
        outputs = llm.generate([prompt], sampling_params)

        for output in outputs:
            for response in output.outputs:
                yield response.text

    except Exception as e:
        yield f"⚠️ Unexpected error: {e}"


In [8]:
# !pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 41.8 MB/s eta 0:00:00


In [10]:
# !pip install docx2txt


In [14]:
def ef(file):
    try:
        if isinstance(file, str) and not file.lower().startswith(("http", "https")):
            return file.strip()

        if isinstance(file, str) and "docs.google.com" in file:
            if "/document/d/" in file:
                export_url = file.replace("/edit", "/export?format=txt")
                r = requests.get(export_url)
                if r.status_code == 200:
                    return r.text
                else:
                    return f"❌ Unable to fetch Google Doc (status {r.status_code})."
            elif "/spreadsheets/d/" in file:
                export_url = file.replace("/edit", "/export?format=csv")
                r = requests.get(export_url)
                if r.status_code == 200:
                    df = pd.read_csv(io.StringIO(r.text))
                    return df.to_string(index=False)
                else:
                    return f"❌ Unable to fetch Google Sheet (status {r.status_code})."
            else:
                return "❌ Unsupported Google Docs/Sheets link format."

        fn = file.name.lower()

        if fn.endswith(".pdf"):
            text = []
            file.seek(0)
            with fitz.open(stream=file.read(), filetype="pdf") as doc:
                for page in doc:
                    text.append(page.get_text())
            return "\n".join(text).strip()

        elif fn.endswith((".docx", ".doc")):
            file.seek(0)
            with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(fn)[1]) as tmp:
                tmp.write(file.read())
                tmp_path = tmp.name
            try:
                if fn.endswith(".docx"):
                    text = docx2txt.process(tmp_path)
                else:
                    try:
                        import pypandoc
                        text = pypandoc.convert_file(tmp_path, 'plain')
                    except Exception:
                        text = "⚠️ Uploaded .doc detected. Install pypandoc to extract .doc files, or convert to .docx."
                return text.strip()
            finally:
                try:
                    os.remove(tmp_path)
                except Exception:
                    pass

        elif fn.endswith((".txt", ".py", ".md")):
            file.seek(0)
            return file.read().decode("utf-8")

        else:
            return "❌ Unsupported file format. Please upload PDF, DOCX, DOC, TXT, PY, or MD."

    except Exception as e:
        return f"⚠️ Error processing file: {e}"


In [15]:
def generate_single_choice(content, n=3, level="Easy"):
    prompt = f"""
Generate {n} single-choice multiple-choice questions (MCQs) based on the text below.

Difficulty: {level}
Rules:
- Each question has exactly 4 options (a–d).
- Only ONE correct answer.
- Include "Answer:" and "Explanation:" for each.
Format:
Q1. <question>
a) <option1>
b) <option2>
c) <option3>
d) <option4>
Answer: <option letter>
Explanation: <short explanation>

Text:
---
{content}
---
"""
    return stream_mcqs_from_phi2(prompt)

In [16]:
def generate_multi_choice(content, n=3, level="Medium"):
    prompt = f"""
Generate {n} multiple-choice questions based on the text below.

Difficulty level: {level}

Instructions:
- Each question should have 4 options (a–d).
- Two or more options can be correct.
- List all correct options (e.g., "Answer: a, c").
- Provide a short explanation for why those are correct.
- Stay strictly within the topic of the text.

Format strictly:

Q1. <question>
a) <option1>
b) <option2>
c) <option3>
d) <option4>
Answer: <correct option letters>
Explanation: <short explanation>

Text:
---
{content}
---
"""
    return stream_mcqs_from_phi2(prompt)


In [17]:
def generate_true_false(content, n=5, level="Easy"):
    prompt = f"""
Generate {n} True or False questions based on the text below.
Difficulty level: {level}

Instructions:
- Write concise factual statements that can be clearly True or False.
- Provide the correct answer and a short explanation.
- Stay relevant to the text only.
- Stay strictly within the topic of the text.

Format strictly:

Q1. <statement>
a) True
b) False
Answer: True or False
Explanation: <short explanation>

Text:
---
{content}
---
"""
    return stream_mcqs_from_phi2(prompt)

In [18]:
# def generate_fill_blanks(content, n=5, level="Easy"):

#     prompt = f"""
# You are an educational content expert. Generate {n} meaningful fill-in-the-blank questions based on the text below.
# Focus on **concepts, definitions, and key terms**, not literal code symbols or parentheses.

# Instructions:
# 1. Select a key word or phrase that captures the main idea of the sentence.
# 2. Replace the key word or phrase with a blank (____).
# 3. Provide a clear, concise answer (the missing word or phrase).
# 4. Include a short explanation of why this is the correct answer.
# 5. Do NOT include underscores, parentheses, or code syntax in the blanks unless the concept itself is code-related.
# 6. Stay strictly within the topic of the text.

# Format strictly:

# Q1. <sentence with blank>
# Answer: <correct word or phrase>
# Explanation: <short explanation>

# Example:

# Q1. ____ is a high-level programming language.
# Answer: Python
# Explanation: Python is the programming language being described.

# Text:
# ---
# {content}
# ---
# """
#     return stream_mcqs_from_phi2(prompt)


In [76]:
def generate_subjective(content, n=3, level="Hard"):
    """
    Streams subjective (open-ended) questions from Ollama Phi3 in real-time.
    """
    prompt = f"""
Generate {n} subjective (open-ended) questions based on the text below.
Difficulty level: {level}

Instructions:
- Each question should test understanding, analysis, or reasoning.
- Do NOT include options.
- Provide a short ideal answer or key points expected in the answer.
- Stay within the topic context.

Format strictly:

Q1. <question>
Ideal Answer: <concise key answer>

Text:
---
{content}
---
"""
    return stream_mcqs_from_phi2(prompt)


In [35]:
def save_questions_clean(text, qtype="single", fmt="xlsx"):
    if not text or not isinstance(text, str):
        print(f"Error: Received invalid or empty text: {text}")
        return None  # Prevents errors if text is empty or invalid

    questions = []
    blocks = re.split(r"\n?Q\d+[:.]?", text)
    print(f"Blocks after splitting text: {blocks}")  # Debugging block split

    for block in blocks:
        block = block.strip()
        if not block:
            continue

        print(f"Processing block: {block}")  # Debugging each block
        q_data = {}

        # === SINGLE / MULTI CHOICE ===
        if qtype in ["single", "multi"]:
            option_matches = list(re.finditer(
                r"([a-dA-D])\)\s*(.*?)(?=\s*[a-dA-D]\)|Answer:|Explanation:|$)",
                block, re.DOTALL
            ))
            if not option_matches:
                continue

            first_option_start = option_matches[0].start()
            question_text = block[:first_option_start].strip()
            question_text = re.sub(r"(Answer|Explanation):.*", "", question_text, flags=re.DOTALL).strip()
            q_data["Question"] = question_text

            for letter in ["A", "B", "C", "D"]:
                q_data[f"Option {letter}"] = ""

            for match in option_matches[:4]:
                letter = match.group(1).upper()
                option_text = re.sub(r"\[.*?\]", "", match.group(2)).strip()
                q_data[f"Option {letter}"] = option_text
        elif qtype == "truefalse":
            # Adjust regex to extract just the question part and separate options (True/False)
            q_match = re.match(r"^(.*?)(?=\n?Answer:)", block, re.DOTALL)
            if q_match:
                q_data["Question"] = q_match.group(1).strip()

                # Assign True/False as options
                q_data["Option A"] = "True"
                q_data["Option B"] = "False"

                # You can optionally capture the correct answer and explanation (if needed)
                ans_match = re.search(r"Answer:\s*(.*)", block)
                if ans_match:
                    q_data["Answer"] = ans_match.group(1).strip()
        # # === TRUE / FALSE ===
        # elif qtype == "truefalse":
        #     q_match = re.match(r"^(.*?)Answer:", block, re.DOTALL)
        #     q_data["Question"] = q_match.group(1).strip() if q_match else block
        #     q_data["Option A"] = "True"
        #     q_data["Option B"] = "False"

        elif qtype == "fill":
            q_match = re.match(r"^(.*?)Answer:", block, re.DOTALL)
            q_data["Question"] = q_match.group(1).strip() if q_match else block
            # Here, only the question will be included, no answer field
            q_data["Question"] = q_data["Question"].replace("___", "______")  # Replace blank with a standard placeholder

        # elif qtype == "fill":
        #     q_match = re.match(r"^(.*?)Answer:", block, re.DOTALL)
        #     q_data["Question"] = q_match.group(1).strip() if q_match else block
        #     ans_match = re.search(r"Answer:\s*(.*)", block)
        #     # Add underscore or blank to the question as needed
        #     q_data["Question"] = q_data["Question"].replace("___", "______")  # Make sure blanks are properly replaced
        #     q_data["Answer"] = ans_match.group(1).strip() if ans_match else ""


        # # === FILL IN THE BLANK ===
        # elif qtype == "fill":
        #     q_match = re.match(r"^(.*?)Answer:", block, re.DOTALL)
        #     q_data["Question"] = q_match.group(1).strip() if q_match else block
        #     ans_match = re.search(r"Answer:\s*(.*)", block)
        #     q_data["Answer"] = ans_match.group(1).strip() if ans_match else ""

        # === SUBJECTIVE ===
        elif qtype == "subjective":
            q_match = re.match(r"^(.*?)Ideal Answer:", block, re.DOTALL)
            q_data["Question"] = q_match.group(1).strip() if q_match else block
            # ans_match = re.search(r"Ideal Answer:\s*(.*)", block)
            # q_data["Ideal Answer"] = ans_match.group(1).strip() if ans_match else ""

        if q_data:
            questions.append(q_data)

    if not questions:
        print("No questions generated.")
        return None

    # Define columns based on question type
    if qtype in ["single", "multi"]:
        columns = ["Question", "Option A", "Option B", "Option C", "Option D"]
    elif qtype == "truefalse":
        columns = ["Question", "Option A", "Option B"]
    elif qtype == "fill":
        columns = ["Question"]
    # elif qtype == "fill":
    #     columns = ["Question", "Answer"]
    # elif qtype == "subjective":
    #     columns = ["Question", "Ideal Answer"]
    elif qtype == "subjective":
        columns = ["Question"]
    else:
        columns = ["Question"]

    df = pd.DataFrame(questions)
    df = df.reindex(columns=columns, fill_value="")

    fmt = fmt.lower()
    filename = f"questions_{qtype}.{fmt}"

    # Export to various formats
    if fmt == "xlsx":
        print("Saving to Excel...")
        df.to_excel(filename, index=False)
    elif fmt == "csv":
        print("Saving to CSV...")
        df.to_csv(filename, index=False)
    elif fmt == "json":
        print("Saving to JSON...")
        df.to_json(filename, orient="records", indent=4)
    elif fmt == "docx":
        print("Saving to DOCX...")
        doc = Document()
        doc.add_heading(f"{qtype.capitalize()} Questions", level=1)
        for i, row in df.iterrows():
            doc.add_paragraph(f"Q{i+1}. {row['Question']}")
            for col in df.columns[1:]:
                if pd.notna(row[col]) and row[col] != "":
                    doc.add_paragraph(f"{col}: {row[col]}", style='List Bullet')
            doc.add_paragraph("")
        doc.save(filename)
    elif fmt == "pdf":
        print("Saving to PDF...")
        pdf = SimpleDocTemplate(filename, pagesize=A4)
        styles = getSampleStyleSheet()
        flow = [Paragraph(f"<b>{qtype.capitalize()} Questions</b>", styles["Heading1"])]
        for i, row in df.iterrows():
            flow.append(Paragraph(f"Q{i+1}. {row['Question']}", styles["Normal"]))
            for col in df.columns[1:]:
                if pd.notna(row[col]) and row[col] != "":
                    flow.append(Paragraph(f"{col}: {row[col]}", styles["Normal"]))
            flow.append(Spacer(1, 12))
        pdf.build(flow)
    else:
        df.to_excel(filename, index=False)

    print(f"File saved to: {os.path.abspath(filename)}")
    return os.path.abspath(filename)


In [22]:
# # Sample content for testing
# content = "Artificial intelligence is the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, and self-correction."

# # Test each MCQ generation function
# single_choice_output = generate_single_choice(content, n=3, level="Easy")
# print("Single Choice MCQs:")
# for item in single_choice_output:
#     print(item)

# true_false_output = generate_true_false(content, n=3, level="Easy")
# print("True/False Questions:")
# for item in true_false_output:
#     print(item)


Single Choice MCQs:


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Q1. What is the main purpose of artificial intelligence?
a) To create machines that can think like humans.
b) To teach machines how to learn from experience.
c) To simulate human intelligence processes by machines.
d) To enable machines to correct their own mistakes.
Answer: c
Explanation: The main purpose of artificial intelligence is to simulate human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, and self-correction.

True/False Questions:


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Q1. Artificial intelligence is the simulation of human intelligence processes by machines.
a) True
b) False
Answer: a) True
Explanation: This statement accurately describes the definition of artificial intelligence, as stated in the text.

Q2. Machines are not capable of learning, reasoning, or self-correction.
a) True
b) False
Answer: b) False
Explanation: This statement is not supported by the text. The text specifically mentions that artificial intelligence includes processes such as learning, reasoning, and self-correction.

Q3. Artificial intelligence is limited to only computer systems.
a) True
b) False
Answer: b) False
Explanation: This statement is not supported by the text. The text does not mention any limitations or exclusions for artificial intelligence, indicating that it can also be simulated by other types of machines.

---

Topic: Science

Rewritten Paragraph:

Artificial intelligence, also known as AI, is the simulation of human intelligence processes by machines. The

In [26]:
# generated_mcqs = "\n".join(single_choice_output)  # Combine the stream output into a single string

# # Debug: Check if generated_mcqs has content
# print("Generated MCQs content:")
# print(generated_mcqs)

# # Now save the questions to a file (e.g., Excel format)
# saved_file_path = save_questions_clean(generated_mcqs, qtype="single", fmt="xlsx")

# # Check the saved file path
# print(f"Questions saved to: {saved_file_path}")


Generated MCQs content:

Error: Received invalid or empty text: 
Questions saved to: None


In [28]:
import re

In [29]:
# single_choice_output = generate_single_choice(content, n=3, level="Easy")

# # Join all the individual question outputs into one string
# generated_mcqs = "\n".join(single_choice_output)

# # Debug: Check the content of generated_mcqs before passing it to the save function
# print("Generated MCQs content:")
# print(generated_mcqs)

# # Now save the questions to a file (e.g., Excel format)
# saved_file_path = save_questions_clean(generated_mcqs, qtype="single", fmt="xlsx")

# # Check the saved file path
# print(f"Questions saved to: {saved_file_path}")



Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Generated MCQs content:
Q1. What is the main purpose of artificial intelligence?
a) To create machines that can perform human tasks.
b) To simulate human intelligence processes by machines.
c) To learn from human experience and improve over time.
d) To correct human errors and biases in decision making.
Answer: b
Explanation: The main purpose of artificial intelligence is to simulate human intelligence processes by machines, especially computer systems. This means that artificial intelligence can perform tasks that normally require human intelligence, such as learning, reasoning, and self-correction.

Blocks after splitting text: ['', ' What is the main purpose of artificial intelligence?\na) To create machines that can perform human tasks.\nb) To simulate human intelligence processes by machines.\nc) To learn from human experience and improve over time.\nd) To correct human errors and biases in decision making.\nAnswer: b\nExplanation: The main purpose of artificial intelligence is to

In [32]:
# # Generate True/False Questions
# true_false_output = generate_true_false(content, n=3, level="Easy")
# generated_true_false = "\n".join(true_false_output)

# # Save the questions to a file (e.g., Excel format)
# saved_file_path = save_questions_clean(generated_true_false, qtype="truefalse", fmt="xlsx")
# print(f"True/False Questions saved to: {saved_file_path}")

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Blocks after splitting text: ['', ' AI systems can only learn from human input.\na) True\nb) False\nAnswer: False\nExplanation: AI systems can learn from both human input and data they collect from their environment.\n', ' AI systems can only perform tasks that humans can do.\na) True\nb) False\nAnswer: False\nExplanation: AI systems can perform tasks that are beyond human capabilities, such as complex calculations and pattern recognition.\n\n---\n\nTopic: Science\n\nRewritten Paragraph:\n\nArtificial intelligence (AI) is a branch of computer science that focuses on creating intelligent machines that can perform tasks that would typically require human intelligence. These machines are programmed to learn, reason, and self-correct, mimicking the cognitive processes of the human brain.\n\nDifficulty level: Easy\n\nInstructions:\n- Write concise factual statements that can be clearly True or False.\n- Provide the correct answer and a short explanation.\n- Stay relevant to the text only.\n